In [1]:
import re
import numpy as np
import pandas as pd
import os, gc
from tqdm.auto import tqdm
from datetime import datetime, timezone, timedelta

import torch
from torch.utils.data import DataLoader,Dataset

c:\Users\kangs\miniconda3\envs\newenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import json
from collections import OrderedDict

In [4]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

In [5]:
import datasets
import transformers

In [6]:
%cd D:/VS_code

D:\VS_code


In [7]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# 토크나이저와 모델 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

model_1 = BartForConditionalGeneration.from_pretrained('./1st')

model_3 = BartForConditionalGeneration.from_pretrained('./3st')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [8]:
# DEVICE 초기화
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"DEVICE:{DEVICE}")

# model을 평가모드로 바꿔서 학습이 일어나지 않도록 하고, DEVICE에 전달하기
model.eval()
model = model.to(DEVICE)
model_1 = model_1.to(DEVICE)
model_3 = model_3.to(DEVICE)

DEVICE:cuda


In [9]:
# 원문 데이터 전처리 하는 부분
def token(text):
    text = re.sub('[^!-~ 가-힣]', '', text)
    text = text.replace('  ', '')
    text = tokenizer.encode(text)

    return text

In [10]:
# 전처리된 데이터 요약하는 부분
def summary(text, model, length = 1.0, num = 4, device = DEVICE):
    text = torch.tensor([text]).to(device)
    text = model.generate(text,  length_penalty=length,  num_beams=num,  max_length=512,  eos_token_id=1)
    text = tokenizer.decode(text.squeeze().tolist(), skip_special_tokens=True)

    return text

In [11]:
####################################################### KBS 스크랩 #######################################################

In [12]:
# KBS에서 긁어올 뉴스 분야
KBS_field = ['정치', '경제', '사회', '문화', 'IT·과학', '국제', '재난·환경', '생활·건강', '스포츠', '연예', '날씨', '이슈']

In [13]:
# KBS 뉴스 크롤링 후 json 파일 형태로 return 하는 함수
def KBS_data(KBS_number):
  # 입력받은 KBS 뉴스 번호로 url에서 html 코드를 그대로 불러오는 부분
  url = "https://news.kbs.co.kr/news/view.do?ncd="+str(KBS_number)
  result = requests.get(url)
  obj = BeautifulSoup(result.content, "html.parser")

  # 불러온 url에 뉴스가 없을 경우(등록일이 없을 경우 뉴스가 없음) None으로 return 하기
  if (obj.find('li', {'class' : 'time'}) == None):
    return None
  # 문장 토큰 갯수가 모델 입력 최대 수인 1026개 이상일 경우 건너뛰기
  text = token((((obj.find('div', {'class' : 'view_con_text'})).text).split('\n'))[1].replace('\t', ''))
  if (len(text) > 1026):
    return None

  # 우리가 필요한 뉴스 분야만 긁어오는 조건문
  if ((((obj.find('li', {'class' : 'time'})).text).split('\n'))[-2] in KBS_field):
    data = OrderedDict()
    
    # 순서대로 썸네일, 신문사, 제목, 링크, 등록시간, 분야, 작가, 요약문(단문, 중문, 장문)을 dictionary 형태로 저장
    data['image'] = obj.head.find('meta', {'itemprop' : 'image'}).get('content')
    data['company'] = 'KBS'
    data['title'] = obj.head.find('meta', {'name' : 'title'}).get('content')
    data['url'] = obj.head.find('meta', {'name' : 'url'}).get('content')
    data['time'] = (((obj.find('li', {'class' : 'time'})).text).split('\n'))[2][3:-3]
    data['field'] = (((obj.find('li', {'class' : 'time'})).text).split('\n'))[-2]
    # 작가의 경우 기사에 따라 존재하지 않을 때도 있음(대표적으로 날씨 뉴스) 이 때 오류 방지를 위해 None 값이 저장되도록 설정
    if (obj.find('p', {'class' : "name"}) == None):
      data['writer'] = None
    else:
      data['writer'] = (((obj.find('p', {'class' : "name"})).text).split('\n'))[1][:-2]
    # 긁어온 데이터에서 원문에 해당하는 부분을 바로 summary 함수에 넣어서 요약문을 저장하도록 설정
    data['text_short'] = summary(text, model_1)
    data['text_middle'] = summary(text, model, length=1.5)
    data['text_long'] = summary(text, model_3)

    return data

In [14]:
####################################################### SBS 스크랩 #######################################################

In [15]:
# SBS에서 긁어올 뉴스 분야
SBS_field = ['정치', '경제', '사회', '국제', '생활·문화', '스포츠', '연예']

In [16]:
# SBS 뉴스 크롤링 후 json 파일 형태로 return 하는 함수
def SBS_data(SBS_number):
  # 입력받은 SBS 뉴스 번호로 url에서 html 코드를 그대로 불러오는 부분
  url = "https://news.sbs.co.kr/news/endPage.do?news_id=N"+str(SBS_number)
  result = requests.get(url)
  obj = BeautifulSoup(result.content, "html.parser")

  # 불러온 url에 뉴스가 없을 경우(등록일이 없을 경우 뉴스가 없음) None으로 return 하기
  if (obj.head.find('meta', {'property' : 'og:updated_time'}) == None):
    return None
  # 문장 토큰 갯수가 모델 입력 최대 수인 1026개 이상일 경우 건너뛰기
  text = token((obj.find('div', {'class' : 'text_area'}).text).replace('\n', ''))
  if (len(text) > 1026):
    return None

  # 우리가 필요한 뉴스 분야만 긁어오는 조건문
  if (obj.head.find('meta', {'itemprop' : 'genre'}).get('content') in SBS_field):
    data = OrderedDict()
    
    # 순서대로 썸네일, 신문사, 제목, 링크, 등록시간, 분야, 작가, 요약문(단문, 중문, 장문)을 dictionary 형태로 저장
    data['image'] = obj.head.find('meta', {'property' : 'og:image'}).get('content')
    data['company'] = 'SBS'
    data['title'] = obj.head.find('meta', {'itemprop' : 'name'}).get('content')
    data['url'] = obj.head.find('meta', {'itemprop' : 'url'}).get('content')
    data['time'] = (obj.head.find('meta', {'property' : 'og:updated_time'}).get('content')[:-9]).replace('T', ' ')
    data['field'] = obj.head.find('meta', {'itemprop' : 'genre'}).get('content')
    # 작가의 경우 기사에 따라 존재하지 않을 때도 있음(대표적으로 날씨 뉴스) 이 때 오류 방지를 위해 None 값이 저장되도록 설정
    if (obj.head.find('meta', {'name' : 'Author'}) == None):
      data['writer'] = None
    else:
      data['writer'] = obj.head.find('meta', {'name' : 'Author'}).get('content')
    # 긁어온 데이터에서 원문에 해당하는 부분을 바로 summary 함수에 넣어서 요약문을 저장하도록 설정
    data['text_short'] = summary(text, model_1)
    data['text_middle'] = summary(text, model, length=1.5)
    data['text_long'] = summary(text, model_3)
    
    return data

In [17]:
####################################################### MBC 스크랩 #######################################################

In [18]:
# MBC에서 긁어올 뉴스 분야
MBC_field = ['정치', '사회', '세계', '경제', '스포츠', 'iMBC 연예']
# MBC 뉴스 링크의 경우, 분야별로 링크에서 장르 부분과, 마지막에 붙는 숫자가 다름
# 따라서 번호뿐만 아니라 어떤 장르의 뉴스를 읽어들일 것인지 함께 함수에서 입력받고,
# 그 분야에 따라 마지막에 붙는 숫자를 변동하기 위해 사전형태로 만들어 둠
MBC_genre = {'politics' : '_36119.html',
             'society' : '_36126.html',
             'world' : '_36133.html',
             'econo' : '_36140.html',
             'sports' : '_36154.html',
             'enter' : '_36161.html'}

In [19]:
# MBC 뉴스 크롤링 후 json 파일 형태로 return 하는 함수
def MBC_data(genre, MBC_number):
  # 입력받은 MBC 뉴스 번호와 장르를 통해 url을 읽어들이고, html 코드를 불러오는 부분
  url = "https://imnews.imbc.com/news/2023/"+genre+"/article/"+str(MBC_number)+MBC_genre[genre]
  result = requests.get(url)
  obj = BeautifulSoup(result.content, "html.parser")

  # 불러온 url에 뉴스가 없을 경우(등록일이 없을 경우 뉴스가 없음) None으로 return 하기
  if (obj.head.find('meta', {'name' : 'nextweb:createDate'}) == None):
    return None
  # 문장 토큰 갯수가 모델 입력 최대 수인 1026개 이상일 경우 건너뛰기
  text = token((((obj.find('div', {'class' : 'news_txt'})).text).replace('\r', ' ')).replace('\n', ''))
  if (len(text) > 1026):
    return None

  # 우리가 필요한 뉴스 분야만 긁어오는 조건문
  if (obj.head.find('meta', {'property' : 'article:section'}).get('content') in MBC_field):
    data = OrderedDict()
    
    # 순서대로 신문사, 제목, 링크, 등록시간, 분야, 작가, 요약문을 dictionary 형태로 저장
    data['image'] = obj.head.find('meta', {'property' : 'og:image'}).get('content')
    data['company'] = 'MBC'
    data['title'] = obj.head.find('meta', {'name' : 'title'}).get('content')
    data['url'] = obj.head.find('meta', {'name' : 'url'}).get('content')
    data['time'] = obj.head.find('meta', {'name' : 'nextweb:createDate'}).get('content')
    data['field'] = obj.head.find('meta', {'property' : 'article:section'}).get('content')
    if (data['field'] == '세계'): data['field'] = '국제'
    # 작가의 경우 기사에 따라 존재하지 않을 때도 있음(대표적으로 날씨 뉴스)
    # 이 때 오류 방지를 위해 None 값이 저장되도록 설정 (MBC의 경우 날씨 기자가 iMBC 연예 같은 신문사 계정일 때도 있음)
    if (obj.head.find('meta', {'name' : 'author'}) == None):
      data['writer'] = None
    else:  
      data['writer'] = obj.head.find('meta', {'name' : 'author'}).get('content')
    # 긁어온 데이터에서 원문에 해당하는 부분을 바로 summary 함수에 넣어서 요약문을 저장하도록 설정
    data['text_short'] = summary(text, model_1)
    data['text_middle'] = summary(text, model, length=1.5)
    data['text_long'] = summary(text, model_3)
    
    return data

In [20]:
####################################################### 실시간 스크랩 #######################################################

In [21]:
# KBS 뉴스 번호
KBS_number = 7680474
# SBS 뉴스 번호
SBS_number = 1007197457
# MBC 분야별 뉴스 번호
MBC_number = {'politics' : 6485254,
             'society' : 6484948,
             'world' : 6485100,
             'econo' : 6485831,
             'sports' : 6487523,
             'enter' : 6484912}

In [25]:
while True:
  # data 스크랩 해오기
  data = KBS_data(KBS_number)

  # data 체크
  if (data == None):
    # 스크랩한 번호부터 100까지 순차적으로 더하면서 사이 번호를 건너뛰고 생성된 뉴스가 있는지 체크
    for count in range(100):
      data = KBS_data(KBS_number + count)
      # 만약 생성된 뉴스가 있다면 KBS_number 값을 해당 뉴스 번호로 저장하고, 반복문 탈출
      if (data != None):
        KBS_number = KBS_number + count
        break
    # 시스템 과열 방지를 위해 1분간 휴식시키는 부분, 작동 체크를 위한 선 출력
    print('Sleep_KBS_N : ', KBS_number)
    #time.sleep(10)
  else: 
    # data가 null 값이 아니라면, json 파일로 뽑아내서 저장함
    # 파일명은 '(KBS_number)_KBS_news.json' 형태로 저장 (KBS_number는 숫자)
    with open('./KBS_news/' + str(KBS_number) + '_KBS_news.json', 'w', encoding='utf-8') as make_file:
      json.dump(data, make_file, ensure_ascii=False, indent='\t')
    print('---------------------------------')
    print('KBS_N : ', KBS_number)
    # 이미 한번 체크한 번호니까 다음 번호부터 체크하도록 +1 하기
    KBS_number += 1
##############################################################################################################
  # data 스크랩 해오기
  data = SBS_data(SBS_number)

  # data 체크
  if (data == None):
    # 스크랩한 번호부터 100까지 순차적으로 더하면서 사이 번호를 건너뛰고 생성된 뉴스가 있는지 체크
    for count in range(100):
      data = SBS_data(SBS_number + count)
      # 만약 생성된 뉴스가 있다면 SBS_number 값을 해당 뉴스 번호로 저장하고, 반복문 탈출
      if (data != None):
        SBS_number = SBS_number + count
        break
    # 시스템 과열 방지를 위해 1분간 휴식시키는 부분, 작동 체크를 위한 선 출력
    print('Sleep_SBS_N : ', SBS_number)
    #time.sleep(10)
  else:
    # data가 null 값이 아니라면, json 파일로 뽑아내서 저장함
    # 파일명은 '(SBS_number)_SBS_news.json' 형태로 저장 (SBS_number는 숫자)
    with open('./SBS_news/' + str(SBS_number) + '_SBS_news.json', 'w', encoding='utf-8') as make_file:
      json.dump(data, make_file, ensure_ascii=False, indent='\t')
    print('---------------------------------')
    print('SBS_N : ', SBS_number)
    # 이미 한번 체크한 번호니까 다음 번호부터 체크하도록 +1 하기
    SBS_number += 1
##############################################################################################################
  # MBC_genre의 key 값별로 data 스크랩 하기 위한 for문 선언
  for genre in MBC_genre:
      data = MBC_data(genre, MBC_number[genre])

      # data 체크
      if (data == None):
        # 스크랩한 번호부터 1000까지 순차적으로 더하면서 사이 번호를 건너뛰고 생성된 뉴스가 있는지 체크
        for count in range(1000):
          data = MBC_data(genre, MBC_number[genre] + count)
          # 만약 생성된 뉴스가 있다면 MBC_number 값을 해당 뉴스 번호로 저장하고, 반복문 탈출 / 변동된 값은 사전을 업데이트 시키며 완료됨
          if (data != None):
            MBC_number[genre] = MBC_number[genre] + count
            break
        # 시스템 과열 방지를 위해 30초간 휴식시키는 부분, 작동 체크를 위한 선 출력 (MBC는 최대 600개의 링크를 체크해야하여 30초만 휴식시키기로 함)
        print('Sleep) Genre, MBC_N : ', genre,', ', MBC_number[genre])
        #time.sleep(10)
      else:
        # data가 null 값이 아니라면, json 파일로 뽑아내서 저장함
        # 파일명은 '(genre)_(MBC_number)_MBC_news.json' 형태로 저장 (genre는 해당 분야 영어명, MBC_number는 숫자)
        with open('./MBC_news/' + str(MBC_number[genre])+'_'+genre+'_MBC_news.json', 'w', encoding='utf-8') as make_file:
          json.dump(data, make_file, ensure_ascii=False, indent='\t')
        print('---------------------------------')
        print('Genre, MBC_N : ', genre,', ', MBC_number[genre])
        # 이미 한번 체크한 번호니까 다음 번호부터 체크하도록 +1 하기 / 다음 분야 링크 체크하는 걸로 넘어가게 됨
        MBC_number[genre] += 1

---------------------------------
KBS_N :  7680454
---------------------------------
SBS_N :  1007197416
---------------------------------
Genre, MBC_N :  politics ,  6485033
---------------------------------
Genre, MBC_N :  society ,  6484931
---------------------------------
Genre, MBC_N :  world ,  6484941
---------------------------------
Genre, MBC_N :  econo ,  6485645
---------------------------------
Genre, MBC_N :  sports ,  6486304
---------------------------------
Genre, MBC_N :  enter ,  6484886
---------------------------------
KBS_N :  7680455
---------------------------------
SBS_N :  1007197417
Sleep) Genre, MBC_N :  politics ,  6485046
---------------------------------
Genre, MBC_N :  society ,  6484932
Sleep) Genre, MBC_N :  world ,  6484982
Sleep) Genre, MBC_N :  econo ,  6485649
Sleep) Genre, MBC_N :  sports ,  6486323
Sleep) Genre, MBC_N :  enter ,  6484896
Sleep_KBS_N :  7680462
---------------------------------
SBS_N :  1007197418
--------------------------------

KeyboardInterrupt: 